In [1]:
import requests
import json
from bs4 import BeautifulSoup
from csv import writer

# Use client credentials
# Additional
client_id1 = 'client_660b4d0cceb4cf9f989acef154caf4ab'
client_secret1 = 'secret_bd749d4e48d050deb1d8873b061456f2'
# jongho api
client_id2 = 'client_e470bd58b93f29f12bf0a584c8ee6b08'
client_secret2 = 'secret_e721ef5d2089efd16ceef38e10d95d1b'
client_id3 = 'client_aba6e5dec04c5d1a2335705504e56729' 
client_secret3 = 'secret_cabc2a80fd7cd09a918f86b5672962cb'
client_id4 = 'client_b74a3bc9756e77bdcd0a2ffb2e430edb'
client_secret4 = 'secret_666cf9388a8462e398aadf7d4d105879'
# philip api
client_id5 = 'client_4253061ce6d6a7a86cc877d9c9dfa776'
client_secret5 = 'secret_2e3838d9fe9ced976a3fa974689a9d9d'
client_id6 = 'client_a89a8ef1b3cf662607f452a01c8f27b1'
client_secret6 = 'secret_a4f96d2c0790ad05fe041eed2c314dfa'
client_id7 = 'client_80e72e5511155fdde9a18e467f829c15'
client_secret7 = 'secret_4e3af2f4c36e901b8f7c0afbda475f32'
# grace api
client_id8 = 'client_406f326e62a400f902b3f719c8593ab2'
client_secret8 = 'secret_6c7dd26c2428d6074e4a6a6b1bfcced3'
# Nuo api
client_id9 = 'client_19c60ac8e6ddcb881b2ea29962cb34d7'
client_secret9 = 'secret_e4bc232ca320af78499287cf3f5477b8'
client_id10 = 'client_80c69b6fa0e2fb021a23daafa80cde7a'
client_secret10 = 'secret_cb18898e2c6674ff84f78c9918a87429'
client_id11 = 'client_e2048fb1cdea5b654551ebe33b30d471'
client_secret11 = 'secret_ffe3749686b362eee3255848bac52932'

client = {}
client['client_id'] = [client_id1, client_id2, client_id3, client_id4, client_id5, client_id6, client_id7, client_id8, client_id9, client_id10, client_id11]
client['client_secret'] = [client_secret1, client_secret2, client_secret3, client_secret4, client_secret5, client_secret6, client_secret7, client_secret8, client_secret9, client_secret10, client_secret11]

def retrive_properties(client_id, client_secret, pageNumber, postCode="", state="VIC", pageSize=200, filename="../../data/raw/API_Raw_PropertiesInfo.csv"):
  scopes = ['api_listings_read']
  auth_url = 'https://auth.domain.com.au/v1/connect/token'
  url_endpoint = 'https://api.domain.com.au/v1/listings/residential/_search'

  # mock client post request in exchange of a one-time access token
  response = requests.post(auth_url, data = {
                          'client_id':client_id,
                          'client_secret':client_secret,
                          'grant_type':'client_credentials',
                          'scope':scopes,
                          'Content-Type':'text/json'
                          })
  json_res = response.json()
  # take the access token each time (can call a page of size 200)
  access_token=json_res['access_token']
  print("Access:", access_token)
  # authentication for current access token
  auth = {'Authorization':'Bearer ' + access_token}
  url = url_endpoint

  # with the current access token, specify the post search query 
  post_fields = {
        "listingType":"Rent", # only interested in Rents
        "pageSize": pageSize, 
        "pageNumber": pageNumber, # 200 per page
        "propertyTypes":"",
        "minBedrooms":1,
        "minBathrooms":1,
        # "maxPrice": 400,
        "sort": {
            "sortKey": "Default"
        },

        "locations":[
          {
            "state": state,
            "region":"",
            "area":"",
            "suburb":"",
            "postCode": postCode, # will iterate by postcodes
            "includeSurroundingSuburbs":False
          }
        ]
  }

  # get the response of a post search, could either be 200 (success) or some errors
  respond = requests.post(url,headers=auth,json=post_fields)
  content = json.loads(respond.text)
  print("Loaded data: " + str(len(content)))
  # quota error or no relevant data can be found on this page, e.g.
  # {'errors': {'searchParameters.Page': ['Cannot page beyond 1000 records.']},
  # 'message': 'The request is invalid.'}
  if ("errors" in content or len(content) == 0):                                              
    print("Limit notice")
    print(content)
    return 1
  # errors occur when the query type is not from PropertyListing
  elif ("type" in content and content["type"] != "PropertyListing"):
    print("Error accurs (No data)")
    print(content)
    return 2

  # Get request for each property ID and extract attributes, each 'i' is 
  # a single property schema in json format
  for i in content:
      #get details
      id = i['listing']['id']
      listing_type = i['listing']['listingType']
      r = i['listing']['propertyDetails']
      street_address=r['displayableAddress']
      suburb=r['suburb']
      postcode=r['postcode']
      area=r['area']
      if('buildingArea' in r and 'landArea' in r):
        building_area = r['buildingArea']
        land_area = r['landArea']
      else:
        building_area = 0
        land_area = 0
      # latitude and longitude might not be available in some instances
      if ('latitude' in r and 'longitude' in r):
        latitude = r['latitude']
        longitude = r['longitude']
      else:
        latitude = None
        longitude = None
      
      property_type=r['propertyType']
      bathrooms=r['bathrooms']
      bedrooms=r['bedrooms']

      if(str(i).find('carspaces'))>0:
          carspaces=i['listing']['propertyDetails']['carspaces']
      else:
          carspaces=0

      price=i['listing']['priceDetails']['displayPrice'] #[1:4]

      if(i['listing']['dateListed']):
          time = i['listing']['dateListed']
      else:
          time = 0

      info = [id, time, listing_type, price, property_type, area, building_area, land_area, 
      bedrooms, bathrooms, carspaces, street_address, suburb, postcode, latitude, longitude]

      # write the relevant info of current property one row at a time
      with open(filename, 'a', newline='') as g:
          thewriter = writer(g)
          thewriter.writerow(info)

  return 0

# retrive_properties(client_id, client_secret, 1)

### Inialise the csv output file with headers

In [2]:

# Write column headers
headers = ['property_id', 'listing_type', 'price', 'time_listed', 'property_type', 'area', 'building_area', 'land_area', 
'bedrooms', 'bathrooms', 'carspaces', 'street_address', 'suburb', 'postcode', 'latitude', 'longitude']
output_csv = open('../../data/raw/API_Raw_PropertiesInfo.csv', 'w')
# create the csv writer object
csv_writer = writer(output_csv)
# first write the headers
csv_writer.writerow(headers)

output_csv.close()

### Rrtrive rental data

In [3]:
# Retrive rental data by postcode
# postcode 3000~4000
count = 0
limit = len(client['client_id'])

for i in range(3678, 4000): # excuted upto (3278, 3677), start from 3678
    # Check available api list
    if count >= limit:
        print("Exceed the download permit")
        break

    # Retrive data
    print("Retriving data from postcode: " + str(i))

    # read data by page (1~5)
    p = 1
    while p < 6:
        if count >= limit:
            break

        client_id = client['client_id'][count]
        client_secret = client['client_secret'][count]

        num = retrive_properties(client_id, client_secret, p, str(i))
        if num == 1:
            # no data found
            break
        elif num == 2:
            # change the new api
            count += 1
            print("using api" + str(count))
        else:
            # continue to the next page
            p += 1

# postcode 8000~9000
for i in range(8000, 9000):
    if count >= limit:
        print("Exceed the download permit")
        break

    print("Retriving data from postcode: " + str(i))

    p = 1
    while p < 6:
        if count >= limit:
            break
        
        client_id = client['client_id'][count]
        client_secret = client['client_secret'][count]

        num = retrive_properties(client_id, client_secret, p, str(i))
        if num == 1:
            break
        elif num == 2:
            count += 1
            print("using api" + str(count))
        else:
            p += 1
    

Retriving data from postcode: 3678
Access: 0f27d4e943929ebdabab07533644c297
Loaded data: 3
Error accurs (No data)
{'type': 'https://developer.domain.com.au/docs/latest/conventions/rate-limiting', 'title': 'Quota Exceeded', 'detail': 'Exceeded rate limit for current period'}
using api1
Access: a241197d4a780c96808b2855409341dd
Loaded data: 3
Error accurs (No data)
{'type': 'https://developer.domain.com.au/docs/latest/conventions/rate-limiting', 'title': 'Quota Exceeded', 'detail': 'Exceeded rate limit for current period'}
using api2
Access: d4d6a92e867d6c6861b77a13c0b5f911
Loaded data: 3
Error accurs (No data)
{'type': 'https://developer.domain.com.au/docs/latest/conventions/access', 'title': 'Not Authorized', 'detail': 'Project [Sandbox] restricted from access to this environment [Primary].'}
using api3
Access: 6f30d7e4b57d6b56e3dbde794147945a
Loaded data: 3
Error accurs (No data)
{'type': 'https://developer.domain.com.au/docs/latest/conventions/access', 'title': 'Not Authorized', 'deta

#### Visualise Property Schema ``(do not run this)``

In [4]:
"""
[
  {
    "type": "Project",
    "listings": [
      {
        "listingType": "NewHomes",
        "id": 2017835505,
        "projectId": 5437,
        "advertiser": {
          "type": "Agency",
          "id": 32584,
          "name": "Roche Group",
          "logoUrl": "https://images.domain.com.au/img/Agencys/32584/logo_32584.jpeg",
          "preferredColourHex": "#1d2c44",
          "bannerUrl": "https://images.domain.com.au/img/Agencys/32584/banner_32584.jpeg",
          "contacts": [
            {
              "name": "Cameron Grove Sales",
              "photoUrl": "https://images.domain.com.au/img/32584/contact_1637430.png?mod=220821-125111"
            }
          ]
        },
        "priceDetails": {
          "displayPrice": "RESERVED"
        },
        "media": [
          {
            "category": "Image",
            "url": "https://bucket-api.domain.com.au/v1/bucket/image/2017835505_1_1_220726_010938-w2880-h2160"
          },
          {
            "category": "Image",
            "url": "https://bucket-api.domain.com.au/v1/bucket/image/2017835505_2_1_220725_035031-w2048-h1536"
          },
          {
            "category": "Image",
            "url": "https://bucket-api.domain.com.au/v1/bucket/image/2017835505_3_1_220725_035031-w1536-h2048"
          },
          {
            "category": "Image",
            "url": "https://bucket-api.domain.com.au/v1/bucket/image/2017835505_4_1_220725_035031-w1536-h2048"
          },
          {
            "category": "Image",
            "url": "https://bucket-api.domain.com.au/v1/bucket/image/2017835505_5_1_220725_035031-w2048-h1536"
          }
        ],
        "propertyDetails": {
          "state": "NSW",
          "features": [
            "AirConditioning",
            "Ensuite",
            "Gas",
            "Bath",
            "Dishwasher",
            "Study",
            "RainwaterStorageTank"
          ],
          "propertyType": "House",
          "allPropertyTypes": [
            "House"
          ],
          "bathrooms": 2,
          "bedrooms": 4,
          "carspaces": 2,
          "unitNumber": "",
          "streetNumber": "37",
          "street": "Estelville Circuit",
          "area": "Lake Macquarie West",
          "region": "Hunter Region",
          "suburb": "CAMERON PARK",
          "postcode": "2285",
          "displayableAddress": "37 Estelville Circuit, Cameron Park",
          "latitude": -32.91141,
          "longitude": 151.59761,
          "landArea": 586,
          "buildingArea": 265,
          "isRural": false,
          "isNew": true,
          "tags": [
            "newdevelopment"
          ]
        },
        "headline": "Cameron Park - Home under construction, amazing views!",
        "summaryDescription": "<b></b><br />Don't miss out on this one of a kind opportunity, this home is currently under construction, you will  see the interior of the home this weekend, Landscaping and Fencing are about to commence. Hope to see you here this weekend.\r\n\r\nTucked a...",
        "hasFloorplan": true,
        "hasVideo": false,
        "labels": [
          "Under offer",
          "Updated",
          "New homes"
        ],
        "dateListed": "2022-05-28T20:15:55",
        "inspectionSchedule": {
          "byAppointment": true,
          "recurring": false,
          "times": []
        },
        "listingSlug": "37-estelville-circuit-cameron-park-nsw-2285-2017835505"
      }
    ],
    "project": {
      "state": "NSW",
      "id": 5437,
      "name": "Cameron Grove Estate",
      "bannerUrl": "https://images.domain.com.au/img/Agencys/devproject/banner_5437_220726_011050",
      "preferredColorHex": "#419a45",
      "logoUrl": "https://images.domain.com.au/img/Agencys/devproject/logo_5437_220726_011050",
      "labels": [
        "New"
      ],
      "displayableAddress": "28 28 BREAKWELL ROAD, CAMERON PARK, NSW 2285",
      "suburb": "CAMERON PARK",
      "features": [
        ""
      ],
      "media": [
        {
          "category": "Image",
          "url": "https://bucket-api.domain.com.au/v1/bucket/image/af4a92fb-1686-479d-a6f7-13e7fc1425b8-w1500-h966"
        }
      ],
      "projectSlug": "project/5437/cameron-grove-estate-cameron-park-nsw"
    }
  }
]

"""

'\n[\n  {\n    "type": "Project",\n    "listings": [\n      {\n        "listingType": "NewHomes",\n        "id": 2017835505,\n        "projectId": 5437,\n        "advertiser": {\n          "type": "Agency",\n          "id": 32584,\n          "name": "Roche Group",\n          "logoUrl": "https://images.domain.com.au/img/Agencys/32584/logo_32584.jpeg",\n          "preferredColourHex": "#1d2c44",\n          "bannerUrl": "https://images.domain.com.au/img/Agencys/32584/banner_32584.jpeg",\n          "contacts": [\n            {\n              "name": "Cameron Grove Sales",\n              "photoUrl": "https://images.domain.com.au/img/32584/contact_1637430.png?mod=220821-125111"\n            }\n          ]\n        },\n        "priceDetails": {\n          "displayPrice": "RESERVED"\n        },\n        "media": [\n          {\n            "category": "Image",\n            "url": "https://bucket-api.domain.com.au/v1/bucket/image/2017835505_1_1_220726_010938-w2880-h2160"\n          },\n        